# movie recomendation

### i will use these datasets from kaggle

TMDB:  https://www.kaggle.com/datasets/tmdb/tmdb-movie-metadata

movie industry: https://www.kaggle.com/datasets/danielgrijalvas/movies

Netflix Movies and TV Shows : https://www.kaggle.com/datasets/shivamb/netflix-shows

In [507]:
import pandas as pd

In [508]:
tmdb_credits = pd.read_csv('TMDB 5000 Movie Dataset/tmdb_5000_credits.csv')
tmdb_movies = pd.read_csv('TMDB 5000 Movie Dataset/tmdb_5000_movies.csv')

In [509]:
tmdb_credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [510]:
tmdb_movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [511]:
tmdb_credits.loc[tmdb_credits["title"].isna() == True]

,movie_id,title,cast,crew


In [512]:
tmdb_movies.loc[tmdb_movies["original_title"].isna() == True]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count


so there is no nan value in tmdb datasets for title 

## convert str to json in tmdb datasets

## warning : after loading our datasets, run these two cell for one time, unless you will get error

In [513]:
tmdb_movies_json = ["genres", "keywords",
                    "production_companies", 
                    "production_countries", 
                    "spoken_languages" ]


for column in tmdb_movies_json:
    tmdb_movies[column] = tmdb_movies[column].apply(json.loads)

In [514]:
tmdb_credits_json = ["cast", "crew"]

for column in tmdb_credits_json:
    tmdb_credits[column] = tmdb_credits[column].apply(json.loads)

# now we will load Netflix dataset

In [515]:
netflix_titles = pd.read_csv("Netflix Movies and TV Shows/netflix_titles.csv")

In [516]:
netflix_titles.head(1)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."


In [517]:
netflix_titles.loc[netflix_titles["title"].isna() == True]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description


# now we will load movie industry dataser

In [518]:
movie_industry = pd.read_csv("movie industry/movies.csv")

In [519]:
movie_industry.head(1)

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime
0,The Shining,R,Drama,1980,"June 13, 1980 (United States)",8.4,927000.0,Stanley Kubrick,Stephen King,Jack Nicholson,United Kingdom,19000000.0,46998772.0,Warner Bros.,146.0


In [520]:
movie_industry.loc[movie_industry["name"].isna() == True]

,name,rating,genre,year,released,score,votes,director,writer,star,country,budget,gross,company,runtime


## alright we have no nan value in our names, lets look at tmdb data

In [521]:
tmdb_credits[:1]

,movie_id,title,cast,crew
0,19995,Avatar,"[{'cast_id': 242, 'character': 'Jake Sully', '...","[{'credit_id': '52fe48009251416c750aca23', 'de..."


In [522]:
tmdb_movies[:1]

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{'name': 'Ingenious Film Partners', 'id': 289...","[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [523]:
tmdb_movies["genres"][0]

[{'id': 28, 'name': 'Action'},
 {'id': 12, 'name': 'Adventure'},
 {'id': 14, 'name': 'Fantasy'},
 {'id': 878, 'name': 'Science Fiction'}]

**as you can see genres are in json data frame, this in not good for easy acsess**
**so for each row we create a list of genres and finally add it as a column to our dataframe**

In [524]:
# Hint: pls look for a lighter and more pandasy way 

# first findout what genres we have in our data


total_row_genres = [] # after running loop in below, this will be a nested list of all genres for all rows

for row in tmdb_movies["genres"][::]:
    row_genres = [] # in each turn of runnig loop, this list will contane geners or each row
    for dict in row:
        row_genres.append(dict["name"])
    total_row_genres.append(row_genres)

In [525]:
total_row_genres[:1] # we will use this list for addin a column in our tmdb movie dataframe

[['Action', 'Adventure', 'Fantasy', 'Science Fiction']]

**now add total_row_genres to our dataframe**

In [526]:
tmdb_movies.insert(loc = 2,
          column = 'eacy acsess genres',
          value = total_row_genres )

In [527]:
tmdb_movies.head(1)

,budget,genres,eacy acsess genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[Action, Adventure, Fantasy, Science Fiction]",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [528]:
tmdb_movies["production_countries"][0]

[{'iso_3166_1': 'US', 'name': 'United States of America'},
 {'iso_3166_1': 'GB', 'name': 'United Kingdom'}]

In [529]:

total_row_country = [] # after running loop in below, this will be a nested list of all genres for all rows

for row in tmdb_movies["production_countries"][::]:
    
    row_country = [] # in each turn of runnig loop, this list will contane geners or each row

    for dict in row:

        row_country.append(dict["name"])
    total_row_country.append(row_country)

In [530]:
# tmdb_movies.drop(columns=['eacy acsess country'], inplace=True)

In [531]:
tmdb_movies.insert(loc = 3,
          column = 'eacy acsess country',
          value = total_row_country )

In [532]:
tmdb_movies[:1]

,budget,genres,eacy acsess genres,eacy acsess country,homepage,id,keywords,original_language,original_title,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...","[Action, Adventure, Fantasy, Science Fiction]","[United States of America, United Kingdom]",http://www.avatarmovie.com/,19995,"[{'id': 1463, 'name': 'culture clash'}, {'id':...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",...,"[{'iso_3166_1': 'US', 'name': 'United States o...",2009-12-10,2787965087,162.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


**now we want to create a new dataframe using unique genres and countries**

In [542]:
unique_genres = []

for genres in total_row_genres:
    
    for genre in genres:
        if genre not in unique_genres:
            unique_genres.append(genre)

# unique_genres

In [534]:
unique_countries = []

for countries in total_row_country:

    for country in countries:
        if country not in unique_countries:
            unique_countries.append(country)

print(unique_countries)

['United States of America', 'United Kingdom', 'Jamaica', 'Bahamas', 'Dominica', 'Czech Republic', 'Poland', 'Slovenia', 'New Zealand', 'Germany', 'China', 'Canada', 'Italy', 'Japan', 'Malta', 'Australia', 'France', 'Belgium', 'India', 'Netherlands', 'Spain', 'United Arab Emirates', 'Hong Kong', 'Taiwan', 'Ireland', 'Morocco', 'Hungary', 'Singapore', 'Norway', 'Sweden', 'South Africa', 'Russia', 'Romania', 'Mexico', 'Monaco', 'Switzerland', 'Pakistan', 'Malaysia', 'Finland', 'Iceland', 'Denmark', 'Tunisia', 'Philippines', 'Bulgaria', 'South Korea', 'Brazil', 'Peru', 'Luxembourg', 'Bosnia and Herzegovina', 'Kazakhstan', 'Portugal', 'Aruba', 'Libyan Arab Jamahiriya', 'Serbia', 'Ukraine', 'Chile', 'Argentina', 'Panama', 'Austria', 'Greece', 'Lithuania', 'Cambodia', 'Thailand', 'Slovakia', 'Israel', 'Fiji', 'Serbia and Montenegro', 'Turkey', 'Nigeria', 'Cyprus', 'Jordan', 'Bolivia', 'Ecuador', 'Colombia', 'Egypt', 'Bhutan', 'Lebanon', 'Kyrgyz Republic', 'Algeria', 'Indonesia', 'Guyana', 'I

In [568]:
count_countries = {}
for item in unique_countries:
    count_countries[item] = 0

# count_countries

In [572]:
genre_country = []
# [ {gener:genre_name, {counted countries for this gener}}, ... ]
for gener in unique_genres:
    genre_country.append({"genre":gener, "country":count_countries})


# genre_country[:1]

In [557]:
len(tmdb_movies["eacy acsess genres"])

4803

In [566]:
total = []
for genre_index in range(len(unique_genres)):

    for row_index in range(len(tmdb_movies["eacy acsess genres"])):
        
        if unique_genres[genre_index] in tmdb_movies["eacy acsess genres"][row_index]:
            for plases in tmdb_movies["eacy acsess countries"][row_index]:
                
           
    

0 Action ['Action', 'Adventure', 'Fantasy', 'Science Fiction']
0 Action ['Adventure', 'Fantasy', 'Action']
0 Action ['Action', 'Adventure', 'Crime']
0 Action ['Action', 'Crime', 'Drama', 'Thriller']
0 Action ['Action', 'Adventure', 'Science Fiction']
0 Action ['Fantasy', 'Action', 'Adventure']
0 Action ['Action', 'Adventure', 'Science Fiction']
0 Action ['Action', 'Adventure', 'Fantasy']
0 Action ['Adventure', 'Fantasy', 'Action', 'Science Fiction']
0 Action ['Adventure', 'Action', 'Thriller', 'Crime']
0 Action ['Adventure', 'Fantasy', 'Action']
0 Action ['Action', 'Adventure', 'Western']
0 Action ['Action', 'Adventure', 'Fantasy', 'Science Fiction']
0 Action ['Science Fiction', 'Action', 'Adventure']
0 Action ['Adventure', 'Action', 'Fantasy']
0 Action ['Action', 'Comedy', 'Science Fiction']
0 Action ['Action', 'Adventure', 'Fantasy']
0 Action ['Action', 'Adventure', 'Fantasy']
0 Action ['Action', 'Adventure']
0 Action ['Adventure', 'Drama', 'Action']
0 Action ['Adventure', 'Action', 

### relation between :


#### genres and production company
#### genres and release_date
#### genres and budget
#### genres and language
#### genres and popularity
#### popularity and production_companies	


# relation between genres and production country

In [409]:
import seaborn as sns

# lets find out fantasy movies are product in which countries
